### Enter Item ID Below of Source and Target Layers

In [1]:
# Source Layer details
source_item_id = '59ea577aed334362bf7106c6771c82cf'
soure_layer_id = 0

# Target layer to load data into
target_item_id = '59ea577aed334362bf7106c6771c82cf'
target_layer_id = 0

## Logging into your ArcGIS Online Organization

In [2]:
from arcgis.gis import GIS
import getpass

In [3]:
arcgis_org_url="https://foretify.maps.arcgis.com/"
username= "foretify_proton"
#password=getpass.getpass("Enter Password: ")
password = 'Zion2023!!'
gis = GIS(arcgis_org_url, username, password, verify_cert=False)
print("Successfully logged in as: " + gis.properties.user.username)

Setting `verify_cert` to False is a security risk, use at your own risk.


Successfully logged in as: foretify_proton


## Functions and Libraries

In [4]:
import os
import sys
import time
import uuid
from configparser import ConfigParser, SafeConfigParser, RawConfigParser
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
from typing import Optional, List, Dict, Text

from arcgis.gis import GIS, Item
from arcgis.features import feature, FeatureSet, FeatureCollection, FeatureLayer
from arcgis.geometry.filters import intersects
from arcgis.geometry import find_transformation, project, SpatialReference


def  append_to_layer(source_data, target_layer, gis=None, matching_field=None, upsert_flag=False):
    """Append a dataframe to a hosted feature layer
    
    Dataframe is converted to a feature collection and uploaded to users content
    in order to be appended to target layer.

    After processing, item is deleted.

    Args:
        source_data ([dataframe]): Source data that will be appended to target layer.
        target_layer ([layer]): Target layer that data is appended to
    """
    try:
        # Create a temporary name based on GUID ID
        temp_name = uuid.uuid4().hex

        # convert source data to feature collection and add to organization
        fs = FeatureSet.from_dataframe(source_data)
        # For cases where just attributes in a target layer are getting updated with no geometry modifications,
        # we will need to ensure the geometryType and spatialReference properties match the target layer in order 
        # to make a feature collection.
        if fs.features[0].geometry_type == 'Table':
            fs.geometry_type = target_layer.properties.geometryType
            fs.spatial_reference = target_layer.properties.extent.spatialReference
        feat_collection = FeatureCollection.from_featureset(fs)  
        temp_fc_properties = {
                            'title': temp_name,
                            'description':'Temporary collection of features.',
                            'tags': 'arcgis python api, pandas, featureCollection, Temp, Temporary',
                            'text': feat_collection._lyr_json,
                            'type':'Feature Collection'
                            }
        temp_fc_layer = gis.content.add(temp_fc_properties)
        print(f"Attempting to append records in {temp_name} on Hosted Feature Layer {target_layer.properties.name}")

        target_layer.append(upload_format='featureCollection',
                        item_id=temp_fc_layer.id,
                        upsert=upsert_flag,
                        skip_updates=False,
                        skip_inserts=False,
                        update_geometry=False,
                        use_globalids=False,
                        rollback=False,
                        upsert_matching_field=matching_field)
    except Exception as e:
        print(f"Unable to append on Hosted Feature Layer {target_layer.properties.name}")
        print(e, exc_info=True)
    finally:
        # Delete temporary item
        temp_fc_layer.delete()


def append_to_table(source_data, target_table, gis=None, matching_field=None, upsert_flag=False):
    """Append a dataframe to a hosted table.
    
    Dataframe is converted to a csv file and saved to the same folder as the log file. File
    is then uploaded to users content in order to be appended to target table.

    After processing, csv file and item are deleted.

    Args:
        source_data ([dataframe]): Source data that will be appended to target layer.
        target_table ([table]): Target layer that data is appended to
    """
    
    try:
        # Create a temporary name based on GUID ID
        temp_name = uuid.uuid4().hex

        # Set path to store files in the same folder as the log file.  
        source_path = os.path.dirname(__file__)
        source_file_path = os.path.join(source_path, f'{temp_name}.csv')

        # convert source data to csv and add to organization
        source_data.to_csv(source_file_path)
        temp_fc_properties = {
                            'title': temp_name,
                            'description':'Temporary collection of features.',
                            'tags': 'arcgis python api, pandas, featureCollection, Temp, Temporary',
                            'type':'CSV'
                            }
        temp_fc_layer = gis.content.add(item_properties=temp_fc_properties, data=source_file_path)
        print(f"Attempting to append records in {temp_name} on Hosted Feature Layer {target_table.properties.name}")
        source_info = gis.content.analyze(item=temp_fc_layer.id, file_type='csv', location_type='none')
    
        target_table.append(upload_format='csv',
                        item_id=temp_fc_layer.id,
                        source_info = source_info['publishParameters'],
                        upsert=upsert_flag,
                        skip_updates=False,
                        skip_inserts=False,
                        update_geometry=False,
                        use_globalids=False,
                        rollback=False,
                        upsert_matching_field=matching_field)
    except Exception as e:
        print(f"Unable to append on Hosted Feature Layer {target_table.properties.name}")
        print(e, exc_info=True)
    finally:
        # Delete temporary csv and item in the event of an exception
        os.remove(source_file_path)
        temp_fc_layer.delete()


def batch_it(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]


def connect(org_url: str, login_name: str, user_password: str, profile_name: Optional[str]=None):
    """Authenticate and connect to an ArcGIS organization. The GIS is used to access, manage
    and modify a users content.

    Args:
        org_url (str): This should be a web address to either an ArcGIS Enterprise portal or to ArcGIS Online in the 
                       form: <scheme>://<fully_qualified_domain_name>/<web_adaptor> (ArcGIS Enterprise example)
        login_name (str): The login user name (case-sensitive).
        user_password (str): If a username is provided, a password is expected. This is case-sensitive.
        profile_name (str, optional): The name of the profile that the user wishes to use to authenticate, if set, 
                                      the identified profile will be used to login to the specified GIS. Defaults to None.
    """
    if profile_name:
        print(f'Attempting to connect with the credential profile, {profile_name}')
        try:
            gis = GIS(profile=profile_name)
            log_profile_info(gis)
            return gis
        except Exception as e:
            login_failed_message = f"Unable to connect to {org_url} with the profile '{profile_name}'. Please check your credentials and try again."
            print(login_failed_message)
            return False
    else:
        print('No profile specified, attempting to connect using username and password')
        try:
            gis = GIS(url=org_url, username=login_name, password=user_password)
            log_profile_info(gis)
            return gis
        except Exception as e:
            login_failed_message = f'Unable to connect to {org_url}. Please check your credentials and try again.'
            print(login_failed_message)
            return False
       
        
def create_intersect_filter_object(filter_df, batch_count=75) -> List:
    """[summary]

    Args:
        filter_df ([type]): Spatial Dataframe that contains a set of features to be filtered. 
        batch_count (int, optional): [description]. Defaults to 75.

    Returns:
        List: A List of :class:`~arcgis.geometry.Geometry` objects
    """
    # List to store intsert filter objects
    intersect_filter_objs = list()

    # identify layer geometry type
    esri_geometry_type = FeatureSet.from_dataframe(filter_df).geometry_type
    if esri_geometry_type == 'esriGeometryPoint':
        geometry_type = 'points'
        update_sets = [filter_df]
    elif esri_geometry_type == 'esriGeometryPolygon':
        # if geometry is polygon, chunk dataframe into lists based on batch count.
        update_sets = list(batch_it(filter_df, batch_count))
        geometry_type = 'rings'

    for edits in update_sets:
        fset = FeatureSet.from_dataframe(edits)
        in_sr = fset.features[0].geometry['spatialReference']
        combined_geom = list()
        combined_geom_dict = dict()
        combined_geom_dict['geometry'] = dict()
        for feat in fset:
            if geometry_type == 'points':
                coords = [[feat.geometry['x'], feat.geometry['y']]]
                combined_geom.extend(coords)
            elif geometry_type == 'rings':
                combined_geom.extend(feat.geometry[geometry_type])
        combined_geom_dict['geometry'][geometry_type] = combined_geom
        combined_geom_dict['geometry']['spatialReference'] = in_sr
        filter_obj = intersects((combined_geom_dict['geometry']), sr=None)
        intersect_filter_objs.append(filter_obj)

    return intersect_filter_objs
        
               
def featureset_to_df(geometry_list, feature_set):
    '''
    Convert a featureset and a list of updated geometry objects 
    into a dataframe.
    :param geometry_list: List - Required 
        List of ArcGIS Python API geometry objects. The list
        of geometries must match the number of features in
        the featureSet. Order is also important whereas the
        first geometry list item must pertain to the first 
        item in the featureSet.
    :param feature_set: List - Required
        List of ArcGIS Python API FeatureSet objects. The list
        of features must match the number of features in
        the geometry list. Order is also important whereas the
        first feature list item must pertain to the first 
        item in the geometry list.
    :return Spatially enabled Dataframe
    '''
    # Feature counter
    cnt = 0

    # Loop through features
    feature_list = list()
    for geom in geometry_list:
        feature_attributes = feature_set.features[cnt].attributes
        feat = Feature(geometry=geom, attributes=feature_attributes)
        feature_list.append(feat)
        cnt += 1
    fset = FeatureSet(feature_list)

    out_df = fset.sdf

    # Ensure datefields are in the correct format
    for f in feature_set.fields:
        if f['type'] == 'esriFieldTypeDate':
            if out_df[f['name']].dtypes != "datetime64[ns]":
                out_df[f['name']] = pd.to_datetime(out_df[f['name']], unit='ms')

    return out_df
        
        
def geometry_based_query(lyr: FeatureLayer, filter_objects: List[Dict]) -> pd.DataFrame:
    """
    Run a spatially enabled query against a layer based on a list of one or more geometry sets

    Args:
        lyr (FeatureLayer): Layer to be queried based on the filter object geometry.
        filter_objects (List): List of dictionaries that contain a set of geometry objects.

    Returns:
        pd.DataFrame: A dataframe that contains a set of unique features queried from the lyr.
    """

    total_dfs = list()
    for filters in filter_objects:
        df = lyr.query(geometry_filter=filters, as_df=True)
        total_dfs.append(df)
    
    # concatenate dataframes together while leaving just unique rows
    unique_df = pd.concat(total_dfs).drop_duplicates().reset_index(drop=True)
    return unique_df


def get_gis_item(item_id: str, gis: GIS) -> Item:
    """ This is a method that retrieves a layer from AGOL or Enterprise by retrieving the layer based
        on the item id and the GIS config object

    Args:
        item_id (str): Item ID of the content that is requested and returned as an item object 
        gis (arcgis.gis.GIS): GIS authentication object that's passed in to access item.  Note, user must
        have access to item in order to fulfill the request.    

    Raises:
        Exception: Message that is returned in regards to unable to access item.

    Returns:
        arcgis.gis.Item: Item content object for the requested item id.
    """
    print(f"Attempting to retrieve layer with item id: {item_id}")
    item = gis.content.get(item_id)

    if not item:
        item_not_found_message = f"Input Item ID Not Found in GIS: {item_id}"
        print(item_not_found_message)
        raise Exception(item_not_found_message)
    else:
        print(f"Successfully got GIS Item ID: {item_id}")
        return item


def log_profile_info(gis):
    '''
    Output print statement that displays gis properties
    '''
    print("Successfully logged into '{}' via the user '{}'".format(
        gis.properties.portalHostname,
        gis.properties.user.username))


def process_edits(feature_layer, data_frame, operation, gis=None, batch_count=20000, matching_field=None, upsert_flag=False):
    """
    Append data to a Push edits from SDF to hosted feature layer.
    Args:
        feature_layer ([type]): Target layer that data is appended to
        data_frame ([type]): Source data that will be appended to target layer.
        operation ([type]): [description] TODO
        batch_count (int, optional): Maximum Number of records in dataframe that that can be in a
                                        set to be appended to target layer. Defaults to 50000.
    """
    print('Processing {} Events.....'.format(len(data_frame)))
    print(f"Running {operation.upper()} on Hosted Feature Layer {feature_layer.properties.name}")

    # Chunk dataframe into lists based on batch count.
    update_sets = list(batch_it(data_frame, batch_count))

    for edits in update_sets:
        try:
            if feature_layer.properties.type == 'Table':
                append_to_table(edits, feature_layer, gis, matching_field, upsert_flag)
            else:
                append_to_layer(edits, feature_layer, gis, matching_field, upsert_flag)
        except:
            print("Unable to run %s on Hosted Feature Layer %s", operation.upper(), feature_layer.properties.name)
            print("Attempted to run %s on data %s", operation.upper(), edits.spatial.to_featureset().features)

            
def query_layer(lyr, fields=None, geometry_flag=True, time_bound=False, geometry_filter_obj=None):
    """Return dataframe based on a layer query."""
    # Set time to record time spent running the query method from the python api.
    start_time = time.time()

    if geometry_filter_obj != None:
        print(f"Running a geometry based query on {lyr.properties.name}")
        df = geometry_based_query(lyr, geometry_filter_obj)
    elif time_bound:
        print("Running time bound query on {}".format(lyr.properties.name))
        clause = time_bound_clause(datetime_field, time_range)
        df =  lyr.query(where=clause, return_geometry=geometry_flag, as_df=True)
    else:
        print("Running query on {}".format(lyr.properties.name))
        df =  lyr.query(return_geometry=geometry_flag, as_df=True)

    print(f'Completed query of {lyr.properties.name} in {round((time.time() - start_time), 2)} seconds returning {len(df)} features')

    # If geometry field is returned + a set of fields, append the geometry field name to the list.
    if geometry_flag and fields is not None:
        shape_field = df.select_dtypes('geometry').columns[0]
        fields.append(shape_field)

    # out_fields parameter in the query method does not work if date fields are restricted.
    # Must run pandas filter method to restrict fields.
    if fields is None:
        return df
    else:
        return df[fields]
    
def reproject(in_df, out_sr):
    """
    Reproject a dataframe to a new spatial reference.
    :param in_df: Spatially enabled Dataframe - Required
        Contains a shape field that contains a list of 
        ArcGIS Python API geometry objects.
    :param out_sr: SpatialReference - Required
        ArcGIS Python API SpatialReference object specifying the
        desired output spatial reference.
    :return Dataframe based on the input dataframe but in the new
        spatial reference.
    """
    # Ensure "nan" Does Not Appear in Aggregate Output Fields
    [in_df[col].replace(np.nan, '', regex=True, inplace=True) for col in list(in_df.columns)]

    # Convert dataframe into a featureset
    feat_set = in_df.spatial.to_featureset()

    # Extract the geometry from FeatureSet
    in_geom_list = [f.geometry for f in feat_set]

    # test to ensure the spatial reference is consistent across all the input geometries
    in_wkid_lst = [geom['spatialReference']['wkid'] for geom in in_geom_list]
    if len(set(in_wkid_lst)) > 1:
        print('All spatial references in the input geometry list must be identical.')
        raise Exception(
            'All spatial references in the input geometry list must be identical.')

    # set the input spatial reference based on the first spatial reference
    in_sr = in_geom_list[0]['spatialReference']

    # determine if a transformation needs to be applied
    transformation_lst = find_transformation(in_sr, out_sr)['transformations']

    # use the geometry service to reproject the geometry list using a transformation if needed
    if len(transformation_lst):
        out_geom_list = project(in_geom_list, in_sr,
                                out_sr, transformation_lst[0])
    else:
        out_geom_list = project(in_geom_list, in_sr, out_sr)

    # ensure the output geometries have the spatial reference explicitly defined
    for geom in out_geom_list:
        geom['spatialReference'] = out_sr

    out_df = featureset_to_df(out_geom_list, feat_set)

    print("Reprojection successful")
    return out_df
    
    
def time_bound_clause(datetime_field: str, time_range: int) -> Text:
    """Create a where statement to look back at a specified time range in minutes. Time is converted to UTC

    Args:
        datetime_field (str): Field name from layer that is of the data type datetime.
        time_range (int): Value in the unit of minutes.

    Returns:
        Text: A string formatted as a complete SQL clause. 
    """
    look_back_time = (datetime.utcnow() - timedelta(hours=0, minutes=time_range)).strftime('%Y-%m-%d %H:%M:%S')
    where_clause = f"{datetime_field} >= timestamp'{look_back_time}'"

    return where_clause

### Grabbing the Layer content from our source and target layers

In [5]:
target_item = get_gis_item(target_item_id, gis)
target_lyr = target_item.layers[0]

source_item = get_gis_item(source_item_id, gis)
source_lyr = source_item.layers[0]

Attempting to retrieve layer with item id: 59ea577aed334362bf7106c6771c82cf
Successfully got GIS Item ID: 59ea577aed334362bf7106c6771c82cf
Attempting to retrieve layer with item id: 59ea577aed334362bf7106c6771c82cf
Successfully got GIS Item ID: 59ea577aed334362bf7106c6771c82cf


In [6]:
source_df = query_layer(source_lyr)
source_df.head()

Running query on Geoenabled_Chats
Completed query of Geoenabled_Chats in 0.39 seconds returning 1 features


,F_id,created,extracted_entity,geocoded_lat,geocoded_long,geocoding_source,message_body,msg_parsed,username,ObjectId,CreationDate,Creator,EditDate,Editor,SHAPE
0,232,2023-12-13 12:15:00,<NA>,35.0,35.0,asdf,asdfasdf,asdfasd,topowright,459,2023-12-11 21:01:53.236000061,foretify_proton,2023-12-11 21:02:20.821000099,foretify_proton,"{""x"": -13549512.913515791, ""y"": 4744556.551021..."


### Adding data to a layer

In [7]:
tmp_df = source_df.iloc[6:15]
tmp_df

,F_id,created,extracted_entity,geocoded_lat,geocoded_long,geocoding_source,message_body,msg_parsed,username,ObjectId,CreationDate,Creator,EditDate,Editor,SHAPE


In [8]:
process_edits(target_lyr, tmp_df, 'add', gis=gis)

Processing 0 Events.....
Running ADD on Hosted Feature Layer Geoenabled_Chats
